In [1]:
pip install selenium requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Chrome 드라이버 경로 설정
chrome_driver_path = 'C:/path/to/chromedriver.exe'
url = 'http://localhost:5000/'

# ChromeOptions 객체 생성
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-extensions")

# 웹 드라이버 시작
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)

# 파일 선택 버튼 자동 클릭 (JavaScript로 클릭)
file_input = driver.find_element(By.ID, 'imageInput')  
driver.execute_script("arguments[0].click();", file_input)

# 파일 업로드
image_folder_path = 'C:/Users/user/Downloads/images/'  # 실제 이미지 파일이 있는 폴더 경로로 수정 가능
for image_file in os.listdir(image_folder_path):
    file_path = os.path.join(image_folder_path, image_file)
    with open(file_path, 'rb') as f:
        file_data = f.read()
    file_input.send_keys(file_path)  # 파일 경로 대신 파일 데이터를 전달
    time.sleep(1)  

# 이미지 전송 버튼 자동 클릭
upload_button = driver.find_element(By.XPATH, "//input[@value='이미지 전송']")  
upload_button.click()

# 웹 페이지가 로딩되는 동안 대기 (5초 기다림)
time.sleep(5)

# 스크래핑하여 이미지 주소 및 설명 추출 및 저장
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

for i, img_tag in enumerate(soup.find_all('img')):
    img_src = urljoin(url, img_tag.get('src'))
    img_data = requests.get(img_src).content
    img_filename = f"downloaded_images/{i + 1}.jpg"
    with open(img_filename, 'wb') as img_file:
        img_file.write(img_data)

    desc_tag = img_tag.find_next('td')
    desc_text_lines = desc_tag.stripped_strings if desc_tag else []
    desc_text = '\n'.join(desc_text_lines)
    desc_filename = f"downloaded_texts/{i + 1}_description.txt"

    print(f"이미지 주소: {img_src}")
    print(f"이미지 저장: {img_filename}")
    print(f"설명 저장: {desc_filename}")

    os.makedirs('downloaded_texts', exist_ok=True)
    
    with open(desc_filename, 'w', encoding='utf-8') as desc_file:
        desc_file.write(desc_text)

    print("=" * 50)

# 브라우저 닫기
#driver.quit()



이미지 주소: http://localhost:5000/static/detects/result_1.jpg
이미지 저장: downloaded_images/1.jpg
설명 저장: downloaded_texts/1_description.txt
이미지 주소: http://localhost:5000/static/detects/result_2.jpg
이미지 저장: downloaded_images/2.jpg
설명 저장: downloaded_texts/2_description.txt
